In [1]:
# ==> Basic Options <==
import psi4
import numpy as np

# Memory & Output File
psi4.set_memory(int(2e9))
psi4.core.set_output_file('output.dat', False)

# Computation options
psi4.set_options({'basis': 'aug-cc-pvdz',
                  'scf_type': 'df',
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

# ==> Define C_1 Water Molecule <==
h2o = psi4.geometry("""
O
H 1 0.96
H 1 0.96 2 104.5
symmetry c1
""")

手动构建最精髓的波函数，只需给定：

- 结构
- 基底


In [7]:
# ==> Build wavefunction for H2O from scratch with Wavefunction.build() <==
h2o_wfn = psi4.core.Wavefunction.build(h2o, psi4.core.get_global_option('basis'))
h2o_wfn.nalpha()

5

现在的波函数空有外壳，没有实际的数据，必须完成一轮计算后才会有数据

In [9]:
# Get the SCF wavefunction & energies for H2O
scf_e, scf_wfn = psi4.energy('scf', return_wfn=True)
print("A float and a Wavefunction object returned:", scf_e, scf_wfn)

A float and a Wavefunction object returned: -76.04123648669083 <psi4.core.RHF object at 0x0000016B1480A030>


Now, we can access information you would expect a wavefunction to carry — basically everything we couldn't before.
Below is summarized several quantities which will be used throughout the modules and tutorials to come.  All these  wavefunction attributes are available after a Hartree–Fock computation; make sure to try them out on our `scf_wfn`!

| Quantity | Function(s) | Description |
|----------|-------------|-------------|
| Orbital Coefficients, **C** | [wfn.Ca()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.Ca "Go to Documentation"), [wfn.Cb()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.Cb "Go to Documentation") | Returns orbital coefficient matrix for $\alpha$ (Ca) or $\beta$ (Cb) orbitals. (Identical for restricted orbitals) |
| Electron Density, **D** | [wfn.Da()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.Da "Go to Documentation"), [wfn.Db()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.Db "Go to Documentation") | One-particle density matrices for $\alpha$ (Da) and $\beta$ (Db) electrons. (Identical for restricted orbitals) |
| Fock Matrix, **F** | [wfn.Fa()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.Fa "Go to Documentation"), [wfn.Fb()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.Fb "Go to Documentation") | Returns the Fock matrix. For wavefunction with unrestricted orbitals, distinct Fock matrices $F^{\alpha}$ and $F^{\beta}$ for $\alpha$ and $\beta$ orbitals, respectively, are created.|
| Basis Set | [wfn.basisset()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.basisset "Go to Documentation") | Returns basis set associated with the wavefunction. |
| $\alpha$ ($\beta$) electrons | [wfn.nalpha()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.nalpha "Go to Documentation"), [wfn.nbeta()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.nbeta "Go to Documentation") | Returns number of $\alpha$ ($\beta$) electrons of the wavefunction. |
| Irreducible Representations | [wfn.nirrep()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.nirrep "Go to Documentation") | Returns number of irreducible representations (number of symmetry elements). Several objects can utilize molecular symmetry in the wavefunction. |
| Occupied Orbitals | [wfn.doccpi()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.doccpi "Go to Documentation") | Returns number of doubly occupied orbitals per irrep in the wavefunction. |
| Psi Variables | [wfn.variables()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.variables "Go to Documentation") | Returns all Psi variables associated with the method which computed the wavefunction. |
| Energy   | [wfn.energy()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.energy "Go to Documentation") | Returns current energy of the wavefunction. |
| Orbital Energies, $\boldsymbol{\epsilon}$ | [wfn.epsilon_a()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.epsilon_a "Go to Documentation"), [wfn.epsilon_b()](http://psicode.org/psi4manual/master/psi4api.html#psi4.core.Wavefunction.epsilon_b "Go to Documentation") | Returns $\alpha$ (a) and $\beta$ (b) orbital energies. (Identical for restricted orbitals) |

Note: The functions returning any of the matrices mentioned above (**C**, **D**, $\boldsymbol{\epsilon}$), actually
return instances of the `psi4.core.Matrix` class (noticing a pattern here?) and not viewable arrays.  Fortunately,
the previous tutorial introduced how to modify these arrays Python-side, using NumPy views created through `np.asarray()`
or `.np`.

The full list is quite extensive; however, this likely comprises the most utilized functions. It should be noted that the "a" stands for alpha and conversely the beta quantities can be accessed with the letter "b". For now let's ensure that all computations have C1 symmetry; molecular symmetry can be utilized in Psi4NumPy computations but adds significant complexity to the code.

In [16]:
# Try out the wavefunction class member functions!
count = 0
for prop in dir(scf_wfn.basisset()):
    if prop.startswith('__'):
        continue
    print(prop)
    count += 1
print('There are %d member functions in the Wavefunction class.' % count)

ao_to_shell
blend
build
compute_phi
construct_from_pydict
ecp_shell
ecp_shell_on_center
function_to_center
function_to_shell
genbas
has_ECP
has_puream
make_filename
max_am
max_function_per_shell
max_nprimitive
molecule
move_atom
n_ecp_core
n_ecp_shell_on_center
n_frozen_core
name
nao
nbf
nprimitive
nshell
nshell_on_center
print_detail_out
print_out
shell
shell_on_center
shell_to_ao_function
shell_to_basis_function
shell_to_center
zero_ao_basis_set
There are 35 member functions in the Wavefunction class.
